<h1> Lecture 14A

Data Science 8, Summer 2021 </h1>

<h3>
<b>
<ul>
<li>Models of Randomness (Chance)   </li><br>
            
<li>Is the model consistent with the Data?  </li><br>
</ul>
</b>
</h3>

In [ ]:
from datascience import *
import numpy as np
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
plots.rcParams["patch.force_edgecolor"] = True

#The following allows porting images into a Markdown window
from IPython.display import Image

SLIDE: Assessing Models

## Dreaming of better days

<h3>Make an array that contains the population proportions: 5% Ender Pearl, and 95% Other.</h3>

In [ ]:
population_proportions = make_array(0.05, .95)
population_proportions

<h3> Now we take 262 samples from the population. </h3>

In [ ]:
sample_proportions(262, population_proportions)

<h4>Run the cell above many times to see how the numbers change, if by much. </h4>

In [ ]:
def proportions():
    return sample_proportions(262, population_proportions)

In [ ]:
proportions()

<h3>We need only the first number in the array&mdash;that is, the fraction of ender pearls we receive in the sample.</h3>

In [ ]:
def ep_proportion():
    return sample_proportions(262, population_proportions).item(0) 

In [ ]:
ep_proportion()

In [ ]:
#Specify the number of trials---that is, the number of times we take a sample
num_simulations = 1000

num_eps = make_array()

for i in np.arange(num_simulations):
    num_ep = ep_proportion() * 262 # Multiplication by 262 converts the fraction to an integer
    num_eps = np.append(num_eps, num_ep)
num_eps

In [ ]:
Table().with_column('Number of Ender Pearls Received', \
                    num_eps)\
        .hist(bins=np.arange(0, 50))

# out of scope (plots the observed value which was 42!)
plots.plot([42,42], [0,0.15], c="r");
plots.ylim([0, 0.15]);

In [ ]:
## Does that seem realistic?
## Questions
## Slides

## Mendel and Pea Flowers ##

In [ ]:
## Mendel had 929 plants, of which 709 had purple flowers
observed_purples = 709 / 929
observed_purples

In [ ]:
predicted_proportions = make_array(.75, .25)
sample_proportions(929, predicted_proportions)

In [ ]:
def purple_flowers():
    return sample_proportions(929, predicted_proportions).item(0) * 100

In [ ]:
purple_flowers()

In [ ]:
num_simulations=1000

purples = make_array()

for i in np.arange(num_simulations):
    new_purple = purple_flowers()
    purples = np.append(purples, new_purple)

In [ ]:
Table().with_column('Percent of purple flowers in sample of 929', purples).hist()

In [ ]:
Table().with_column('Discrepancy in sample of 929 if the model is true', abs(purples- 75)).hist()

<h3>How close was Mendel's sample to the model's prediction?</h3>

In [ ]:
abs(observed_purples * 100 - 75)

<h1> Lecture 14B

Data Science 8, Summer 2021 </h1>

<h3>
<b>
<ul> 
<li>Comparing Distributions  </li><br>

<li>Hypothesis Testing  </li><br>
</ul>
</b>
</h3>

In [ ]:
from datascience import *
import numpy as np
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
plots.rcParams["patch.force_edgecolor"] = True

#The following allows porting images into a Markdown window
from IPython.display import Image

<h2> Haribo Gummy Bears </h2>

<h3> In this case, we have more than two categories. How would we calculate a test statistic for that? </h3>

In [ ]:
bears = Table().with_columns(
    'Flavor', make_array('Strawberry', 'Lemon', 'Orange', 'Raspberry', 'Pineapple'),
    'Expected', make_array(0.2, 0.2, 0.2, 0.2, 0.2),
    'Observed', make_array(15/60, 10/60, 7/60, 17/60, 11/60)
)

bears

In [ ]:
bears.barh('Flavor')

In [ ]:
model = make_array(0.2, 0.2, 0.2, 0.2, 0.2)
model

In [ ]:
simulation = sample_proportions(60, model)
simulation

In [ ]:
bears.with_column("Simulated", simulation).barh("Flavor")

<h2> Distance Between Distributions </h2>

In [ ]:
# Earlier, the difference between observed purple flowers
# and the expected values (75%) was our statistic.
#
# In this case, we need to understand how each of the 5 categories
# differs from its expected value according to the model.

diffs = bears.column('Observed') - bears.column('Expected')
bears_with_difference = bears.with_column('Difference', diffs)
bears_with_difference

In [ ]:
np.sum(bears_with_difference.column('Difference'))

This is zero, for all intents and purposes. 

In [ ]:
np.sum(abs(bears_with_difference.column('Difference')))/2

Why do we divide by 2?

In [ ]:
example1 = Table().with_columns(
    'Example', make_array('Category 1', 'Category 2', 'Category 3'),
    'Expected', make_array(1/3, 1/3, 1/3),
    'Observed', make_array(1/3, 1/3, 1/3)
)
example1

In [ ]:
sum(abs(example1.column('Observed') -  example1.column('Expected')))

In [ ]:
example2 = Table().with_columns(
    'Example', make_array('Category 1', 'Category 2', 'Category 3'),
    'Expected', make_array(0, 1, 0),
    'Observed', make_array(1, 0, 0)
)
example2

In [ ]:
sum(abs(example2.column('Observed') -  example2.column('Expected')))

In [ ]:
bears_with_difference

In [ ]:
np.sum(abs(bears_with_difference.column("Difference")))

In [ ]:
0.05 + 0.0833333 #sum of positive entries

In [ ]:
-0.0333333 - 0.0833333 - 0.0166667 # sum of negative entries

In [ ]:
np.sum(abs(bears_with_difference.column('Difference')))/2

## Total Variation Distance

In [ ]:
def tvd(dist1, dist2):
    return sum(abs(dist1 - dist2))/2

In [ ]:
# The TVD of our observed data from their expected values
# assuming the model is true
observed_stat = tvd(bears.column('Observed'), bears.column('Expected'))
observed_stat

Same value we got previously. 

<h3>Let's do the same comparison with the simulated distribution:</h3>

In [ ]:
model = make_array(0.2, 0.2, 0.2, 0.2, 0.2)

In [ ]:
sample_proportions(60, model)

In [ ]:
# The TVD of a model simluation from its expected values
tvd(sample_proportions(60, model), bears.column('Expected'))

In [ ]:
def simulated_tvd():
    return tvd(sample_proportions(60, model), model)

In [ ]:
tvds = make_array()

num_simulations = 10000
for i in np.arange(num_simulations):
    new_tvd = simulated_tvd()
    tvds = np.append(tvds, new_tvd)
tvds

In [ ]:
title = 'Simulated TVDs (if model is true)'

Table().with_column(title, tvds).hist()
print('Observed TVD: ' + str(observed_stat))